# Data exploration and clean up

## Imports and loading

In [148]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import os

In [149]:
RAW_DATA_FILE = "data/raw/raw_player_data_2023.csv"

filename = RAW_DATA_FILE


def load():
    player_data = pd.read_csv(filename)

    print(player_data.shape)
    
    return player_data

In [150]:
player_data = load()

player_data.head()

(91672, 88)


,Unnamed: 0,UID,Name,Rec,DOB,Inf,Club,Based,Nat,Height,...,Pres,Loy,Inj Pr,Imp M,Dirt,Amb,Ada,Cons,Cont,Media Handling
0,0,2002078863,Scott King,- - -,10/9/2004 (17 years old),Ama,Colorado International,U.S.A. (Mountain Division),USA,"5'9""",...,14,9,11,10,15,11,7,14,11,Media-friendly
1,1,2002078833,José González,- - -,23/10/2004 (17 years old),Ctr,NC Fusion U23,U.S.A. (South Atlantic Division),USA,"5'4""",...,14,9,3,12,14,11,6,9,8,Media-friendly
2,2,2002078815,Chase Crane,- - -,6/12/2004 (17 years old),Ctr,NC Fusion U23,U.S.A. (South Atlantic Division),USA,"6'2""",...,9,11,2,8,15,13,10,8,9,Level-headed
3,3,2002078813,Emmanuel Araiza,- - -,30/11/2004 (17 years old),Ctr,Oly Town FC,U.S.A. (Northwest Division),USA,"6'1""",...,12,10,4,13,12,10,13,10,10,Media-friendly
4,4,2002078799,Yang Xiaofeng,- - -,30/6/2004 (17 years old),Ctr,Oly Town FC,U.S.A. (Northwest Division),CHN,"6'0""",...,9,11,14,7,7,6,11,7,5,Level-headed


## Data Cleaning and Preprocessing

### Column names

In [151]:
player_data.columns = player_data.columns.str.strip()

### Replacing dashes with NaN

In [152]:
def clean_columns(val):
    if pd.isnull(val) or str(val) == "-":
        return np.nan
    return val    

# apply
for column in player_data.columns:
    player_data[column] = player_data[column].apply(clean_columns)

### Datetime conversion

In [153]:
# Only process DOB if it exists
if 'DOB' in player_data.columns:
    # Only modify if DOB is not already datetime
    if not np.issubdtype(player_data['DOB'].dtype, np.datetime64):
        # Step 1: Extract the date part (everything before the space)
        player_data['Date'] = player_data['DOB'].astype(str).str.extract(r'(^[\d/]+)')

        # Step 2: Convert to datetime safely
        player_data['DOB'] = pd.to_datetime(player_data['Date'], format='%d/%m/%Y', errors='coerce')

# Compute age (only for valid DOBs)
if 'DOB' in player_data.columns:
    today = dt.datetime.strptime("01/01/2022", '%d/%m/%Y')
    player_data['Age'] = (today - player_data['DOB']).dt.days // 365

### Clean currency

In [154]:
# Some currency columns like 'Transfer Value' are stored as strings. We try to clean them.
def clean_currency(val):
    if pd.isnull(val) or val == "Not for Sale":
        return np.nan
    # Remove any currency symbols or commas
    val = str(val).replace('$', '').replace(',', '').strip()
    try:
        return float(val)
    except:
        return np.nan

if 'Transfer Value' in player_data.columns:
    player_data['Transfer Value Clean'] = player_data['Transfer Value'].apply(clean_currency)

### Clean weights

In [155]:
def clean_weight(val):
    if pd.isnull(val):
        return np.nan
    # Remove any currency symbols or commas
    val = str(val).replace('kg', '').strip()
    try:
        return float(val)
    except:
        return np.nan

if 'Weight' in player_data.columns:
    player_data['Weight'] = player_data['Weight'].apply(clean_weight)

### Clean heights

In [156]:
def height_to_cm(height):
    if pd.isnull(height):
        return np.nan
    
    # Remove spaces and handle strings like 5'6", 5'6, etc.
    height = str(height).strip().replace('"', '').replace(' ', '')
    
    # Match pattern like 5'6
    match = re.match(r"^(\d+)'(\d+)?$", height)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2)) if match.group(2) else 0
        total_cm = round((feet * 12 + inches) * 2.54, 1)
        return total_cm
    
    return np.nan  

# Apply conversion
if 'Height' in player_data.columns:
    player_data['Height'] = player_data['Height'].apply(height_to_cm)

## Dropping unwanted columns

In [158]:
# Remove unwanted index column if present
if 'Unnamed: 0' in player_data.columns:
    player_data.drop(columns=['Unnamed: 0'], inplace=True)

In [159]:
player_data.drop(columns=["Rec","Date", "Transfer Value"], inplace=True, errors="ignore")

## Grouping up fields into a hierarchy

In [160]:
column_groups = {
    'General': [
        'UID', 'Name', 'Rec', 'DOB', 'Inf', 'Club', 'Based', 'Nat',
        'Height', 'Weight', 'Age', 'Position', 'Transfer Value',
        'Preferred Foot', 'Left Foot', 'Right Foot'
    ],
    'Matches': [
        'Imp M', 'Caps', 'AT Apps', 'AT Gls', 'AT Lge Apps', 'AT Lge Gls',
        'Team', 'Yth Apps', 'Yth Gls'
    ],
    'Physical': [
        'Acc', 'Str', 'Sta', 'Pac', 'Nat.1', 'Jum', 'Bal', 'Agi'
    ],
    'Mental': [
        'Wor', 'Vis', 'Tea', 'OtB', 'Ldr', 'Fla', 'Cnt', 'Cmp', 'Bra', 'Ant',
        'Agg', 'Dec', 'Det', 'Pos'
    ],
    'Goalkeeping': [
        'Thr', 'TRO', 'Ref', 'Pun', '1v1', 'Kic', 'Han', 'Ecc', 'Cmd', 'Aer', 'Com'
    ],
    'Technical': [
        'Tec', 'Tck', 'Pen', 'Pas', 'Mar', 'L Th', 'Lon', 'Hea', 'Fre',
        'Fir', 'Fin', 'Dri', 'Cro', 'Cor'
    ],
    'Other': [
        'Vers', 'Temp', 'Spor', 'Prof', 'Pres', 'Loy', 'Dirt', 'Amb', 'Ada', 'Cons'
    ],
    'Injury': [
        'Rc Injury', 'Inj Pr'
    ],
    'Media': [
        'Media Description', 'Media Handling', 'Cont'
    ]
}

# --- Flatten the mapping into a list of tuples for MultiIndex ---
multi_cols = []
for category, cols in column_groups.items():
    for col in cols:
        multi_cols.append((category, col))

# --- Create the MultiIndex ---
multi_index = pd.MultiIndex.from_tuples(multi_cols)

# --- Align with existing columns in player_data ---
# Keep only columns that actually exist in your DataFrame
valid_cols = [col for _, col in multi_cols if col in player_data.columns]

# Rebuild the filtered MultiIndex based on existing columns
filtered_multi_cols = [(grp, col) for grp, col in multi_cols if col in player_data.columns]
filtered_index = pd.MultiIndex.from_tuples(filtered_multi_cols)

# --- Reorder and apply the MultiIndex ---
player_data = player_data[valid_cols]
player_data.columns = filtered_index

# --- Verify ---
player_data.head()

General                                                           \
          UID             Name        DOB  Inf                    Club   
0  2002078863       Scott King 2004-09-10  Ama  Colorado International   
1  2002078833    José González 2004-10-23  Ctr           NC Fusion U23   
2  2002078815      Chase Crane 2004-12-06  Ctr           NC Fusion U23   
3  2002078813  Emmanuel Araiza 2004-11-30  Ctr             Oly Town FC   
4  2002078799    Yang Xiaofeng 2004-06-30  Ctr             Oly Town FC   

                                                            ... Other       \
                              Based  Nat Height Weight Age  ...   Loy Dirt   
0        U.S.A. (Mountain Division)  USA  175.3   65.0  17  ...     9   15   
1  U.S.A. (South Atlantic Division)  USA  162.6   56.0  17  ...     9   14   
2  U.S.A. (South Atlantic Division)  USA  188.0   77.0  17  ...    11   15   
3       U.S.A. (Northwest Division)  USA  185.4   74.0  17  ...    10   12   
4       U.S.A. (Northwest Division)  CHN  182.9   72.0  17  ...    11    7   

                  Injury                     Media                       
  Amb Ada Cons Rc Injury Inj Pr  Media Description  Media Handling Cont  
0  11   7   14       NaN     11       Young winger  Media-friendly   11  
1  11   6    9       NaN      3   Young midfielder  Media-friendly    8  
2  13  10    8       NaN      2   Young goalkeeper    Level-headed    9  
3  10  13   10       NaN      4  Young center back  Media-friendly   10  
4   6  11    7       NaN     14  Young center back    Level-headed    5  

[5 rows x 85 columns]

## Mapping contract info

In [161]:
inf_map = {
    'Ama': 'Amateur',
    'Ctr': 'Contracted',
    'Wnt': 'Wanted',
    'Yth': 'Youth',
    'Loa': 'On Loan',
    'Trn': 'Transfer Listed',
    'Inj': 'Injured',
    'FrA': 'Free Agent',
    'Set': 'Set for Transfer',
    'Lst': 'Listed',
    'Una': 'Unavailable',
    'nEU': 'Non-EU Player',
    'Rst': 'Rested',
    'Ret': 'Retiring',
    'Wp': 'Work Permit Required',
    'Sus': 'Suspended',
    'Frt': 'Fractured/Injured',
    'IPR': 'In Pre-Registration',
    'Dnt': 'Do Not Transfer'
}

# player_data[('General', 'Inf Description')] = (
#     player_data['General']['Inf'].map(inf_map)
# )


In [162]:
# All injured or suspended players
# player_data[player_data['General']['Inf'].isin(['Inj', 'Sus'])]


## All sub tables

In [163]:
player_data["General"]

,UID,Name,DOB,Inf,Club,Based,Nat,Height,Weight,Age,Position,Preferred Foot,Left Foot,Right Foot
0,2002078863,Scott King,2004-09-10,Ama,Colorado International,U.S.A. (Mountain Division),USA,175.3,65.0,17,AM (R),Right,Fairly Strong,Very Strong
1,2002078833,José González,2004-10-23,Ctr,NC Fusion U23,U.S.A. (South Atlantic Division),USA,162.6,56.0,17,M (C),Right,Reasonable,Very Strong
2,2002078815,Chase Crane,2004-12-06,Ctr,NC Fusion U23,U.S.A. (South Atlantic Division),USA,188.0,77.0,17,GK,Right,Fairly Strong,Very Strong
3,2002078813,Emmanuel Araiza,2004-11-30,Ctr,Oly Town FC,U.S.A. (Northwest Division),USA,185.4,74.0,17,D (C),Right,Reasonable,Very Strong
4,2002078799,Yang Xiaofeng,2004-06-30,Ctr,Oly Town FC,U.S.A. (Northwest Division),CHN,182.9,72.0,17,D (LC),Left,Very Strong,Reasonable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91667,20030405,Hunter Gorskie,1991-06-27,NaN,Monterey Bay,U.S.A. (USSL-C Western Conference),USA,180.3,73.0,30,"D (RC), DM",Right,Reasonable,Very Strong
91668,19074835,Marcos Vinícius,1991-06-27,NaN,Foz do Iguaçu,Brazil (Lower Division),BRA,190.5,84.0,30,D (C),Left Only,Very Strong,Weak
91669,14023371,Germán Pezzella,1991-06-27,NaN,Real Hispalis,Spain (First Division),ARG,188.0,82.0,30,D (C),Right,Reasonable,Very Strong
91670,37001809,Jordy Clasie,1991-06-27,NaN,AZ,Netherlands (Eredivisie),NED,170.2,68.0,30,"D (C), DM, M (C)",Right,Reasonable,Very Strong


In [164]:
player_data["Technical"]

,Tec,Tck,Pen,Pas,Mar,L Th,Lon,Hea,Fre,Fir,Fin,Dri,Cro,Cor
0,13,5,3,8,3,4,6,6,6,8,9,15,6,3
1,15,6,6,15,10,4,7,6,8,13,5,6,6,5
2,7,3,1,5,2,1,3,1,6,5,2,3,1,2
3,10,9,5,8,8,7,7,10,3,8,4,4,5,6
4,10,7,3,7,6,6,3,5,3,6,2,4,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91667,9,11,7,10,12,7,5,11,5,8,7,6,7,6
91668,7,10,7,8,9,4,1,14,2,9,2,2,3,4
91669,10,14,7,10,16,8,7,16,7,8,7,7,8,7
91670,14,13,11,15,12,8,10,7,11,14,6,13,9,9


In [165]:
player_data["Mental"]

,Wor,Vis,Tea,OtB,Ldr,Fla,Cnt,Cmp,Bra,Ant,Agg,Dec,Det,Pos
0,11,5,11,10,12,12,6,6,8,8,8,9,12,4
1,13,10,17,9,11,9,8,12,6,8,10,9,11,9
2,5,3,9,2,1,3,5,5,6,9,4,9,16,6
3,7,7,11,6,13,3,8,8,16,10,6,9,12,9
4,14,5,12,3,2,2,7,5,17,6,14,10,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91667,14,7,13,6,11,4,13,9,14,11,8,9,11,11
91668,8,6,5,7,9,5,12,8,9,11,7,14,15,9
91669,14,12,14,9,16,10,14,15,15,16,15,14,15,15
91670,15,16,15,13,14,14,15,14,16,14,17,15,16,14


In [166]:
player_data["Physical"]

,Acc,Str,Sta,Pac,Nat.1,Jum,Bal,Agi
0,12,5,7,10,12,4,12,13
1,9,4,9,10,8,1,10,12
2,7,8,6,6,15,12,13,11
3,10,7,9,13,9,10,10,13
4,12,5,9,8,7,9,6,9
...,...,...,...,...,...,...,...,...
91667,12,10,14,11,12,10,10,11
91668,10,12,9,10,12,14,12,10
91669,11,14,13,12,13,16,14,11
91670,10,8,11,10,14,8,14,13


In [167]:
player_data["Goalkeeping"]

,Thr,TRO,Ref,Pun,1v1,Kic,Han,Ecc,Cmd,Aer,Com
0,2,1,1,1,3,2,2,3,1,3,3
1,3,2,1,3,3,4,3,2,2,1,3
2,9,11,16,13,11,7,13,1,11,9,11
3,3,3,1,2,3,1,3,1,3,3,2
4,1,2,3,3,2,3,2,4,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...
91667,3,1,1,1,2,3,3,1,1,3,1
91668,1,2,3,1,3,2,1,1,2,2,3
91669,2,3,1,2,2,1,3,3,3,2,1
91670,2,2,1,1,1,1,2,2,2,3,1


In [168]:
player_data["Other"]

,Vers,Temp,Spor,Prof,Pres,Loy,Dirt,Amb,Ada,Cons
0,12,8,13,10,14,9,15,11,7,14
1,9,10,8,11,14,9,14,11,6,9
2,12,17,12,15,9,11,15,13,10,8
3,14,17,17,10,12,10,12,10,13,10
4,8,11,12,13,9,11,7,6,11,7
...,...,...,...,...,...,...,...,...,...,...
91667,12,18,11,11,10,11,6,11,12,16
91668,14,14,9,13,11,13,5,11,15,10
91669,10,14,15,15,12,15,13,12,14,11
91670,11,13,14,14,16,16,12,14,7,17


In [169]:
player_data["Injury"]

,Rc Injury,Inj Pr
0,NaN,11
1,NaN,3
2,NaN,2
3,NaN,4
4,NaN,14
...,...,...
91667,NaN,9
91668,NaN,13
91669,Torn knee ligaments,10
91670,NaN,9


In [170]:
player_data["Media"]

,Media Description,Media Handling,Cont
0,Young winger,Media-friendly,11
1,Young midfielder,Media-friendly,8
2,Young goalkeeper,Level-headed,9
3,Young center back,Media-friendly,10
4,Young center back,Level-headed,5
...,...,...,...
91667,Center back,Media-friendly,8
91668,Center back,Level-headed,6
91669,Center back,Level-headed,12
91670,Defensive midfielder,Level-headed,8


In [171]:
player_data["Matches"]

,Imp M,Caps,AT Apps,AT Gls,AT Lge Apps,AT Lge Gls,Team,Yth Apps,Yth Gls
0,10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
91667,13,0,201,8,201,8,NaN,NaN,NaN
91668,5,0,17,NaN,17,NaN,NaN,NaN,NaN
91669,13,29,257,14,257,14,Main,7,NaN
91670,14,17,308,13,308,13,NaN,12,1
